In [1]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets
import numpy as np
import pandas as pd

data =datasets.load_iris()


x, y = data.data, data.target
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.3,random_state=0)

kernel = 1.0 * RBF(1.0)

class MY_RBFN:

    def __init__(self, X, y, tX, ty, num_of_classes,std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.std_from_clusters = std_from_clusters

    def find_center(self,X):
        return np.array(X)
    
    def equil_distance(self,x1, x2):
        sum = 0
        for i in range(len(x1)):
            sum += (x1[i] - x2[i]) ** 2
        return np.sqrt(sum)

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def interpolitation_matrix(self, X, centroids, std_list):
        matrix_list = []
        for x in X:
            matrix_list.append([1 / np.exp(-self.equil_distance(x, c) / s ** 2) for (c, s) in zip(centroids, std_list)])
        return np.array(matrix_list)

    def fit(self):
        self.centroids= self.find_center(self.X)
        dMax = np.max([self.equil_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
        self.std_list = np.repeat(dMax / np.sqrt(2 * self.X),self.number_of_classes)
        RBF_X = self.interpolitation_matrix(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.interpolitation_matrix(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

    def performace_score(self):
        
        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Our Custome Algorithm Accuracy:  ', len(np.where(diff == 0)[0]) / len(diff))

In [2]:
RBF_CLASSIFIER = MY_RBFN(train_x, train_y, test_x, test_y, num_of_classes=3, std_from_clusters=True)

RBF_CLASSIFIER.fit()
RBF_CLASSIFIER.performace_score()

gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(x, y)
print("Python Built Algorithm  Accuracy:",gpc.score(x, y))

Our Custome Algorithm Accuracy:   0.9777777777777777
Python Built Algorithm  Accuracy: 0.9866666666666667
